In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as pls
import scipy

In [ ]:
class GARCH_1_1:
    
    '''
    Initialize log returns with returns passed in
    '''
    def __init__(self, returns):
        self.log_returns = np.log(returns)
        
        
    '''
    Returns the variance expression of a GARCH(1,1) process.
    '''
    def garch_filter(self, omega, alpha, beta):
               
        # Length of log_returns
        length = len(self.log_returns)
        
        # Initializing an empty array
        sigma_sq = np.zeros(length)
        
        # Filling the array, if i == 0 then uses the long term variance.
        for i in range(length):
            if i == 0:
                sigma_sq[i] = omega / (1 - alpha - beta)
            else:
                sigma_sq[i] = omega + alpha * self.log_returns[i-1]**2 + beta * sigma_sq[i-1]
        
        return sigma_sq
    
    '''
    Defines the log likelihood sum to be optimized given the parameters.
    '''
    def garch_loglikehihood(self, parameters):
        
        length = len(self.log_returns)
        
        sigma_sq = self.garch_filter(parameters)
        
        log_likelihood = - np.sum(-np.log(sigma_sq) - self.log_returns**2 / sigma_sq)
        
        return log_likelihood
    
    ''' 
    Optimizes the log likelihood function and returns estimated coefficients
    '''
    def garch_optimization(self):
        
        # Parameters initialization
        parameters = [.1, .05, .92]
        
        # Parameters optimization, scipy does not have a maximize function, so we minimize the opposite of the equation described earlier
        opt = scipy.optimize.minimize(self.garch_loglikehihood, parameters,
                                     bounds = ((.001,1),(.001,1),(.001,1)))
        
        variance = .01**2 * opt.x[0] / (1 - opt.x[1] - opt.x[2])   # Times .01**2 because it concerns squared returns
        
        return np.append(opt.x, variance)
    